Name : Ingwer Ludwig<br>
NRP : 5025201259<br>

In [1]:
#Make directory input and output
!mkdir input-5025201259
!mkdir output-5025201259
!mkdir output-5025201259/path
!mkdir output-5025201259/checkpoint

In [2]:
#Download json files
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=19TQUEIE7mQYNPWuep4RYnKp03litnoqV' -O ./input-5025201259/news-1.json
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=19Wa1h30-lWtF0z6UGg7WIBsZeLK2Gz1W' -O ./input-5025201259/news-2.json
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=19Xl5pbiww7yZ1Pt_tbhkLcZgJMSPj__Y' -O ./input-5025201259/news-3.json




--2023-04-14 03:01:22--  https://docs.google.com/uc?export=download&id=19TQUEIE7mQYNPWuep4RYnKp03litnoqV
Resolving docs.google.com (docs.google.com)... 142.251.16.113, 142.251.16.138, 142.251.16.101, ...
Connecting to docs.google.com (docs.google.com)|142.251.16.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ddgps0nlitto7l246h1mkotqreiovls5/1681441275000/07765186029080978570/*/19TQUEIE7mQYNPWuep4RYnKp03litnoqV?e=download&uuid=f56e8362-ba10-4154-a26a-b6ebcf91d0ce [following]
--2023-04-14 03:01:22--  https://doc-0s-88-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ddgps0nlitto7l246h1mkotqreiovls5/1681441275000/07765186029080978570/*/19TQUEIE7mQYNPWuep4RYnKp03litnoqV?e=download&uuid=f56e8362-ba10-4154-a26a-b6ebcf91d0ce
Resolving doc-0s-88-docs.googleusercontent.com (doc-0s-88-docs.googleusercontent.com)... 172.253.62.132, 2607:f8b0:

In [3]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=a28e369bc6b49a34a5f1cf32c2226e0275ab655a9b0da9f03d044c6497a56e14
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


## Initiate the Spark session

In [4]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [5]:
import json

# Open the JSON file in read mode
with open('./input-5025201259/news-1.json') as f:
    for line in f:
        data = json.loads(line)
        json_string = json.dumps(data, indent=4)
        print(data)

# Print the Python object
print(data)

{'link': 'https://www.huffpost.com/entry/covid-boosters-uptake-us_n_632d719ee4b087fae6feaac9', 'headline': 'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters', 'category': 'U.S. NEWS', 'short_description': 'Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.', 'authors': 'Carla K. Johnson, AP', 'date': '2022-09-23'}
{'link': 'https://www.huffpost.com/entry/american-airlines-passenger-banned-flight-attendant-punch-justice-department_n_632e25d3e4b0e247890329fe', 'headline': 'American Airlines Flyer Charged, Banned For Life After Punching Flight Attendant On Video', 'category': 'U.S. NEWS', 'short_description': "He was subdued by passengers and crew when he fled to the back of the aircraft after the confrontation, according to the U.S. attorney's office in Los Angeles.", 'authors': 'Mary Papenfuss', 'date': '2022-09-23'}
{'link': 'https://www.huffpost.com/entry

The columns are :<br>
link<br>
headline<br>
category<br>
short_description<br>
authors<br>
date<br>

## Define the data structure

In [6]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType

schema = StructType([\
        StructField("link", StringType(), True),\
        StructField("headline", StringType(), True),\
        StructField("category", StringType(), True),\
        StructField("short_description", StringType(), True),\
        StructField("authors", StringType(), True),\
        StructField("date", DateType(), True),\
])

## Determine the folder location

In [7]:
#The folder is our previously made folder
df = spark.readStream.schema(schema).json("./input-5025201259/")

In [8]:
df.printSchema()

root
 |-- link: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- category: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- date: date (nullable = true)



## Do the aggregation step

In [ ]:
groupDF = df.select("headline").groupBy("headline").count()

groupDF.printSchema()

## Start the streaming task

In [ ]:
result = groupDF.writeStream\
  .format("console")\
  .option("path", "./output-5025201259/path")\
  .option("checkpointLocation", "./output-5025201259/checkpoint")\
  .outputMode("complete")\
  .start()


result.awaitTermination()

The output can be seen in the file explorer inside google collab